In [70]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch import nn
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

In [71]:
from utils import get_flatten_iab_tags

iab_tags = get_flatten_iab_tags('baseline/IAB_tags.csv')
iab_tags[:5]

['Транспорт',
 'Транспорт: Типы кузова автомобиля',
 'Транспорт: Типы кузова автомобиля: Грузовой автомобиль',
 'Транспорт: Типы кузова автомобиля: Седан',
 'Транспорт: Типы кузова автомобиля: Универсал']

In [72]:
class VideoDataset(Dataset):
   def __init__(self, descriptions, tags, tokenizer, max_len):
	   self.descriptions = descriptions
	   self.tags = tags
	   self.tokenizer = tokenizer
	   self.max_len = max_len
	   self.max_tags = len(iab_tags)

   def __len__(self):
	   return len(self.descriptions)

   def __getitem__(self, idx):
	   description = str(self.descriptions[idx])
	   tag = self.tags[idx]

	   inputs = self.tokenizer(description, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')
	   
	   tag_tensor = np.zeros(self.max_tags)
	   for t in tag:
		   if t < self.max_tags:
			   tag_tensor[t] = 1.0

	   return {
		   'input_ids': inputs['input_ids'].flatten(),
		   'attention_mask': inputs['attention_mask'].flatten(),
		   'labels': torch.tensor(tag_tensor, dtype=torch.float)
	   }

In [73]:
class VideoTaggerModel(BertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        outputs = super(VideoTaggerModel, self).forward(input_ids, attention_mask=attention_mask, **kwargs)
        logits = outputs.logits

        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            # Применим sigmoid для логитов
            loss = loss_fct(logits, labels)
            return loss, logits
        
        return logits

In [74]:
def split_tags(tags: str):
	splitted_tags = list(filter(bool, map(str.strip, str(tags).replace('  ', ' ').split(','))))
	return splitted_tags


df = pd.read_csv('baseline/train_data_categories.csv')
df['splitted_tags'] = df['tags'].apply(split_tags)
df.head()

,video_id,title,description,tags,splitted_tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира]
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...,[События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма...","[Массовая культура: Отношения знаменитостей, М..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура","[Транспорт, Спорт: Автогонки, Массовая культура]"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира,[Массовая культура: Юмор и сатира]


In [75]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 256

tags_encoder = {tag: idx for idx, tag in enumerate(iab_tags)}
df['encoded_tags'] = df['splitted_tags'].apply(
	lambda tags: np.array([tags_encoder.get(tag, 0) for tag in tags])
)

In [76]:
train_df, test_df = train_test_split(df, test_size=0.1)
train_dataset = VideoDataset(train_df['description'].values, train_df['encoded_tags'].tolist(), tokenizer, max_len)
test_dataset = VideoDataset(test_df['description'].values, test_df['encoded_tags'].tolist(), tokenizer, max_len)

In [77]:
model = VideoTaggerModel.from_pretrained('bert-base-uncased', num_labels=len(iab_tags))

training_args = TrainingArguments(
   output_dir='./results',
   num_train_epochs=3,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   warmup_steps=500,
   weight_decay=0.01,
   logging_dir='./logs'
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset
)

Some weights of VideoTaggerModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Step,Training Loss
